# Свёрточные нейросети и POS-теггинг

Попробуем на практике разобраться, как применять свёрточные нейросети к задачам обработки текста. Разбираться мы будем на примере задачи определения частей речи слов, по-английски она называется "POS-tagging" (part of speech tagging). Зачем эта задача нужна и в чём сложности — мы увидим в процессе

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

Эта задача относится к области лингвистического анализа, то есть анализа структуры текстов. Основная сложность в этой задаче, обычно, заключается в том, чтобы правильно определять часть речи для омонимов, то есть для слов, которые пишутся одинаково, но, на самом деле, имеют разные части речи. Другая сложность заключается в том, чтобы правильно определять части речи для неизвестных слов, то есть каких-нибудь неологизмов или специальной редкой лексики (научной, например). 

**Модели, которые получились ниже — уже достаточно неплохие, хотя промышленные POS тэггеры работают с гораздо более высоким качеством (на уровне 0.97 f-мер), у нас где-то 0.9.**

- здесь 4 модели:
  - время обучения одной модели на полном наборе данных:
    - до 0,89: **30-40 минут**
    - до сходимости `5e-3`: **1.5 часа**
    - gpu: GF760gtx 2Gb

In [1]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

In [2]:
# !pip install pyconll
# !pip install spacy_udpipe

Что делает load_ext autoreload 

По умолчанию, если модуль уже импортировался и встречается его повторный импорт, то модуль ищется в `sys.modules`, и повторный импорт ничего не дает. На такой случай по-классике есть `importlib.reload`. Расширение `autoreload` интерактивного питона просто снимает необходимость следить за изменениями импортированных модулей, само посмотрит время изменения, если надо перезагрузит. Великая вещь.

In [3]:
# расширение IPython для переимпорта
# зачем? потому что есть самописная dlnlputils, если ее править на ходу, то автоимпорт полезен
%load_ext autoreload    
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import gc  # будем удалять большие промежуточные объекты, а то память как не в себя кушает (привет языковой сервер микрософт)

from sklearn.metrics import classification_report

import numpy as np

import pyconll      # для загрузки размеченного корпуса

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

Сделаем пару ускоряшек

In [4]:
# проверка работоспособности, модели не сохраняются
QUICK_RUN = True     
# не обучать, а взять последнюю обученную модель
LOAD_LAST_MODEL = True     
# доля исходных датасетов для обучения/валидации
SAMPLE_SIZE = 1             
# число эпох обучения НС
NUM_TRAIN_EPOCHS = 10       

if QUICK_RUN:
    LOAD_LAST_MODEL = False
    SAMPLE_SIZE = 0.05
    NUM_TRAIN_EPOCHS = 1

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

Используем размеченный корпус, который называется "SynTag Rus":
- размечен лингвистами
- содержит разметку:
  - по частям речи
  - по нормальным формам слов
  - синтаксическую разметку 
- предназначен для того, чтобы настраивать и проверять методы лингвистического анализа текстов:
  -  морфологического разбора
  -  синтаксического разбора
  
Просто скачиваем с гитхаба


In [5]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# !wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
# !wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

Разметка в этом корпусе представлена в [формате CoNLL](https://gitlab.com/mwetoolkit/mwetoolkit2-legacy/-/blob/master/test/filetype-samples/corpus.conll) — это достаточно распространённый формат для того, чтобы хранить аннотированные деревья и разную лингвистическую разметку. Он используется не только в этом корпусе — он достаточно популярный.

In [6]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

In [7]:
# pyconll это пакет про язык, масочную идексацию не понимает
full_train = full_train[ : round(len(full_train) * SAMPLE_SIZE)]
full_test = full_test[ : round(len(full_test) * SAMPLE_SIZE)]
print(f"Train size:\t{len(full_train)}\nTest size:\t{len(full_test)}")

Train size:	1226
Test size:	445


Данные состоят из предложений, предложения из токенов. Что такое токен:

In [8]:
t = full_train[2][1]
[(attr, t.__getattribute__(attr)) for attr in ['form'] + t.__slots__]

[('form', 'приемной'),
 ('id', '2'),
 ('_form', 'приемной'),
 ('lemma', 'приемная'),
 ('upos', 'NOUN'),
 ('xpos', None),
 ('feats',
  {'Animacy': {'Inan'},
   'Case': {'Loc'},
   'Gender': {'Fem'},
   'Number': {'Sing'}}),
 ('head', '6'),
 ('deprel', 'obl'),
 ('deps', {'6': ('obl', 'в', 'loc', None)}),
 ('misc', {})]

Что такое предложение:

In [9]:
for token in full_train[2]:
    print(f"{token.form}\t{token.upos}")

В	ADP
приемной	NOUN
его	PRON
с	ADP
утра	NOUN
ожидали	VERB
посетители	NOUN
,	PUNCT
-	PUNCT
кое-кто	PRON
с	ADP
важными	ADJ
делами	NOUN
,	PUNCT
а	CCONJ
кое-кто	PRON
и	PART
с	ADP
такими	DET
,	PUNCT
которые	PRON
легко	ADV
можно	ADV
было	AUX
решить	VERB
в	ADP
нижестоящих	ADJ
инстанциях	NOUN
,	PUNCT
не	PART
затрудняя	VERB
Семена	PROPN
Еремеевича	PROPN
.	PUNCT


Статистика предложений и  токенов (используется для задания размеров тензоров)

In [10]:
MAX_SENT_LEN = max(max(len(sent) for sent in full_train), max(len(sent) for sent in full_test)) 
MAX_ORIG_TOKEN_LEN = max(max(len(token.form) for sent in full_train for token in sent), 
                         max(len(token.form or [0]) for sent in full_test for token in sent))
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 70
Наибольшая длина токена 22


Примеры предложений

In [11]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

Решать задачу определения части речи мы будем с помощью **свёрточных нейросетей.**  
Будем использовать нейросети, которые принимают на вход номера **отдельных символов**

Это вполне оправдано, потому что часть речи во многом определяется именно структурой слова, наличием суффиксов, окончаний определённого вида... 

Если бы мы работали на уровне отдельных токенов, то мы бы просто не могли анализировать структуру слов, нам бы приходилось просто запоминать, что такое-то слово — это существительное, другое слово это просто глагол... такого типа задача совсем из другой плоскости. Поэтому следующий этап обработки корпуса — это **построение словаря символов**:
- словарь - просто `dict`
- добавлен фиктивный символ:
  - нужно для выравнивания векторных представлений
    - нужно для удобной заливки на GPU

In [12]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 79
[('<PAD>', 0), (' ', 1), ('о', 2), ('а', 3), ('е', 4), ('и', 5), ('.', 6), ('л', 7), ('н', 8), ('т', 9)]


In [13]:
del train_char_tokenized    # надо чистить от ненужного
gc.collect()

0

- аналогично кодируем части речи

In [14]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

Переводим все в тензоры, чтобы скормить торчу.

- `TensorDataset` торча - принимает на вход списки тензоров и их меток: тензоры идентификаторов символов и идентификаторы меток частей речи.

`pos_corpus_to_tensor` (пришлось прикостылить немного, часть тестового набора не читается):

- создает эти тензоры,
- входной тензор трехмерный (предложения, токены, символы)
- в символьных тензорах добавлено две колонки с 0:
  - далее в них будет указываться положение N-граммы в токене (начало, середина, конец), чтобы НС могла учитывать, что
    - Одна и та же последовательность символов может быть как частью суффикса, так и частью приставки, при этом неся разную функцию
    - Для определения части речи в русском языке конец слова (окончания и суффиксы) важнее, чем середина и начало слова

In [15]:
train_inputs, train_labels, err1 = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels, err2 = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

print(f"Train read errors:\t{err1}\nTest read errors:\t{err2}")

Train read errors:	0
Test read errors:	0


In [16]:
del full_train, full_test   # больше не нужны
gc.collect()

0

Входной тензор:

- прямоугольный тензор, 
- каждая строчка этого тензора представляет один токен и содержит номера символов, которые в этом токене используются, в том порядке, в котором они встречались в самом токене
- стартовый ноль в токене нужен для того, чтобы указать нейросети, что это — начало токена. Всё, что после последнего символа — заполняется нулями
- количество значащих элементов в каждой такой строчке у нас отличается, есть короткие и длинные токены. 

In [17]:
train_inputs[1][:5]

tensor([[ 0, 39,  3, 25,  3,  7, 19,  8,  5, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0,  2, 24,  7,  3, 11,  9,  8,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 16, 17, 10,  3, 12,  7,  4,  8,  5, 18,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 11, 12, 18, 21,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 37,  4, 15,  4,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0]])

Выходной (целевой) тензор:
- одномерный тензор, просто список чисел
- каждое число представляет номер тэга соответствующего токена

In [18]:
train_labels[1]

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [19]:
del train_inputs, test_inputs       # уже погрузили в тензоры, больше не нужны, а лейблы нужны будут на валидации
gc.collect()

0

In [20]:
print(f"Train tokens:\t{train_labels.count_nonzero()}\nTest tokens:\t{test_labels.count_nonzero()}")

Train tokens:	17263
Test tokens:	8557


## Вспомогательная свёрточная архитектура

Вспомогательный нейросетевой модуль, включает:
- свертки
- функцию активации
- дропауты (методы регуляризации нейронной сети для предотвращения переобучения)

`forward`:
- модуль (`ModuleList`) состоит из **слоев**
- для ускорения сходимости к выходу каждого слоя прибавляется вход
  - такая штука называется **skip connection**
  - без skip connection мы можем делать нейросеть глубины 5-9 
    - иначе сеть не будет сходиться, т.к. большая размерность и небольшое изменение градиента уводит очень далеко в пространстве
  - со skip connection сеть будет (должна) сходиться с произвольной глубиной, градиент двинется не так далеко от входа
  - такая штука похожа на простой вариант [ResNet](https://en.wikipedia.org/wiki/Residual_neural_network) (Residual neural network)

**Слои**:
- реализуются объектом торча `Sequential`:
  - сам по себе является конвеером
  - берет кортеж слоев и последовательно по ним передает выходы на входы
-  **Первый слой**:
   -  одномерная свертка (основной вариант для текста), число каналов на входе равно числу каналов на выходе
   -  размер ядра по умолчанию равен 3
   -  добавлен паддинг (фиктивные элементы по краям), чтобы тензор вообще не менялся в размере
- **Второй слой**:
  - дропаут (dropout): нужен, чтобы НС меньше переобучалась, для этого в начале обучения зануляет случайные ячейки тензора, а когда НС уже обучена - ничего не делает
  - параметр - это вероятность зануления (в торче по умолчанию 0.5), у нас по-умолчаю 0.0 (**отключено**)
- **Третий слой**:
  - функция активации, тут это `LeakyReLU` (часто это — неплохой выбор для текста)

ReLU:
$$f(x) =
  \begin{cases}
    x, & x > 0\\
    0, & \text{otherwise}
  \end{cases}$$
    
LeakyReLU:
$$f(x) =
  \begin{cases}
    x, & x > 0\\
    \alpha x, & \text{otherwise}
  \end{cases}$$

В LeakyReLU по умолчанию $\alpha = 0.01$. Дает маленький ненулевой градиент в том случае, когда нейрон достиг насыщения и неактивен.

In [21]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, dropout=0.0, dilation=1, conv_layer=nn.Conv1d):
        super().__init__()
        pd = int((dilation * (kernel_size - 1)) / 2)    # паддинг должен учитывать разреженность окна
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=pd, dilation=dilation),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Про дропаут

Стоит отметить момент про коррекцию весов у обученной с dropout'ом сети. В train-time нейроны (их выходы) отключаются с вероятностью $p$, в test-time они уже обучены присутствуют все (генерируют выходы), следовательно на следующие нейроны приходит сигнал более высокий, чем был на этапе обучения, поэтому веса связей необходимо скорректировать.

**Для этого, исходящие веса нейронов, которые обучались в режиме dropout умножаются на $(1-p)$.** Ну или бывает реализация, когда меняются не веса, а добавляется коррекция непосредственно на значения входов следующего слоя нейронов, результат тот же.

В pytorch перед обучением модели ее необходимо перевести в режим обучения (критически важно для слоев `DropOut` и `BatchNorm`): 

    model.train()

Перед оценкой качества/применением модели необходимо вызвать

    model.eval()

Тут мы это делаем ниже внутри `train_eval_loop`

## Предсказание частей речи на уровне отдельных токенов

Будет предсказывать метки частей речи только по векторам токенов, не используя контекст их употребления в предложениях. Случаи "Сорок сорок сидело на дереве" или "Три да три будет шесть или три да три будет дырка?" не осилит.

`forward`

1. Делаем **эбмеддинги**:
- получим переменные, представляющие форму исходного тензора
- схлопываем первое и второе измерения, чтобы получить двухмерный тензор
  - так мы забываем о том, что токены у нас были как-то объединены в предложения
- `nn.Embedding` - слой, который для каждого символа выдает вектора ембеддингов
- "транспонируем" (permute) получившийся 3д-тензор, чтобы можно было его передать на свертку в соответствии с **конвенцией торча о порядке размерностей**:
  - размер батча
  - количество признаков у каждого элемента (тут это размер эмбеддинга)
  - остальные размерности элементов
    - тексты одномерные, поэтому тут у нас просто длина токена (в картинках будет 2д, ну и т.д.)

Теперь char_embeddings содержит вектора для отдельных символов (без информации о контексте этих символов, просто факт, что они есть). В начале обучения они инициализируются случайным шумом.

2. Передаем эти вектора в **backbone** сетку (в роли которого простенький ResNet, который описан выше)
   - она (StackedConv1d) пройдет по случайным эмбеддингам символов сверткой с ядром размера 3 и вот мы уже получили признаки с учетом контекста символов в токене (`features`)
   - **НО** тэги нам нужно предсказывать не для каждого символа, а для каждого токена
   - поэтому мы передаем эти векторы признаков в агрегатор (pooling)
  
3. Используем **pooling** — в данном случае это max pooling:
   - допустим у нас есть матрица (строки - символы в токене, столбцы - признаки), max pooling выдаст вектор по длине признаков, где каждое значение это максимум по столбцу (берется признак самого значимого символа в токене для каждого символа)
   - пройдя по тензорам всех токенов получим уже двумерный тензор (`global_features`), каждая строчка этого тензора представляет отдельный токен

4. Передаем в выходной модуль **out** (в роли которого полносвязный слой `nn.Linear`, что по сути есть линейный (мульти)классификатор, только тензор вместо матрицы $y=xA^T+b$ ). Он должен навесить метки на вектора признаков:
   - он выдает также 2д тензор, но его размерность не длина эмбеддинга, а "количество меток частей речи" (по количеству токенов в тензоре)
   - меняем форму этого тензора, преобразуем его в трёхмерный, просто возвращая разбивку на предложения (max_sent_len)
   - транспонируем для того, чтобы порядок измерений соответствовал порядку измерений в исходном тензоре "tokens"

Физический смысл того, что мы сейчас описали, заключается в том, чтобы рассмотреть все возможные N-граммы символов, которые встречаются в каждом токене, и по ним попробовать определить часть речи. 

Благодаря тому, что основная наша нейросеть (backbone) содержит "skip connections", N-граммы, которые учитываются этой нейросетью, по сути, имеют различную длину. Например, если мы используем размер ядра свёртки, равный 3, то первый блок учитывает трёхграммы, второй блок уже учитывает пятиграммы, а третий — семиграммы, соответственно. При этом, благодаря тому, что есть "skip connection", информация о трёхграммах не теряется, она пробрасывать до самого конца. **Здорово, правда?!**
   

In [22]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

#### Пояснения к алгоритму, почему тензор батча трехмерный

В начале мы фиксируем максимальную длину предложения $SentenceSize$:

$$Batch = \left( \begin{matrix} w_{1,1} & w_{1,2} & ... & w_{1,S} \\ w_{2,1} & w_{2,2} & ... & w_{2,S} \\w_{3,1} & w_{3,2} & ... & w_{3,S} \end{matrix} \right) \in \mathbb{R} ^ {Batch \times SentenceSize}$$

Каждое слово состоит из букв + тэги начала и конца слова, все они имеют свой embedding vector:

$$w_{1,1} = \left( \begin{matrix} c_{1,1} & c_{1,2} & ... & c_{1,k} \end{matrix} \right) = \left( \begin{matrix} \left[ \begin{matrix} e_{1,1} \\ e_{2,1} \\ ... \\ e_{E,1} \end{matrix} \right] & \left[ \begin{matrix} e_{1,2} \\ e_{2,2} \\ ... \\ e_{E,2} \end{matrix} \right] & ... & \left[ \begin{matrix} e_{1,k} \\ e_{2,k} \\ ... \\ e_{E,k} \end{matrix} \right] \end{matrix} \right) \in \mathbb{R} ^ {EmbedSize \times WordSize}$$ 

Таким образом $Batch$ представляет собой тензор размерности 4:

$$\newline Batch = \left( \begin{matrix} \left( \begin{matrix} \left[ \begin{matrix} e_{1,1} \\ e_{2,1} \\ ... \\ e_{E,1} \end{matrix} \right] & \left[ \begin{matrix} e_{1,2} \\ e_{2,2} \\ ... \\ e_{E,2} \end{matrix} \right] & ... & \left[ \begin{matrix} e_{1,k} \\ e_{2,k} \\ ... \\ e_{E,k} \end{matrix} \right] \end{matrix} \right) & ... & ... \\ ... & ... & ... \\... & ... & ... \end{matrix} \right) \in \mathbb{R} ^ {Batch \times SentenceSize \times EmbedSize \times WordSize}$$

Первая нейросеть принимает на вход одно слово и не учитывает его контекст (соседние слова), поэтому для нее мы переделываем батч:

1. Транспонируем  embedding vectors для каждого слова

2. Один элемент батча - одно слово

$$\newline Batch' = \left( \begin{matrix} \left( \begin{matrix} \left[ \begin{matrix} e_{1,1} & e_{2,1} & ... & e_{E,1} \end{matrix} \right] \\ \left[ \begin{matrix} e_{1,2} & e_{2,2} & ... & e_{E,2} \end{matrix} \right] \\ ... \\ \left[ \begin{matrix} e_{1,k} e_{2,k} & ... & e_{E,k} \end{matrix} \right] \end{matrix} \right) \\ ... \end{matrix} \right) \in \mathbb{R} ^ {Batch * SentenceSize \times WordSize \times EmbedSize}$$

Создаем экземпляр созданного нейросетевого модуля с параметрами, в т.ч.:
- эмбеддинг 64
- три сверточный слоя
- размер ядра 3
- дропаут 0.3

Три слоя с ядром 3 с учетом skip_connection охватят N-граммы размера 3, 5 и 7.

В нейросети получилось 47 тыс. параметров, это микросетка по современным меркам.

*На 760gtx 10 эпох считались более получаса, но что характерно, у авторов курса на эпоху уходит 120 сек., а старичок 760gtx выдает 180-190 сек. что наверно очень даже не плохо, ведь врядли авторы на таком старье считали*.

Более того, в комментариях пишут, что в гуглоколабе на карточке Tesla P100-PCIE-16GB время обучения одной эпохи примерно в два раза выше авторсого, т.е. около 300 сек. (мож он там шарится на несколько клиентов?). **А то выходит, 760gtx до сих пор актуальна**

In [23]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), 
                                          embedding_size=64, layers_n=3, kernel_size=3, 
                                          dropout=0.3)
                                          
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 43282


Функция потерь т.к. это многомерная классификация - кроссэнтропия:

- кроссэнтропия батча токенов - это вектор кроссэнетропий размера батча, а `F.cross_entropy` возвращает среднее по этому вектору

In [24]:
if LOAD_LAST_MODEL:
    # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
    try:
        msg = single_token_model.load_state_dict(torch.load('./models/single_token_pos.pth'))
        print("Loading pretrained: ", msg)
    except RuntimeError as e:
        print("Конфигурация сети поменялась, сохраненная модель не подходит")
        raise e
else:
    torch.cuda.empty_cache()
    
    (val_loss,
    single_token_model) = train_eval_loop(single_token_model,
                                          train_dataset,
                                          test_dataset,
                                          F.cross_entropy,
                                          lr=5e-3,
                                          epoch_n=NUM_TRAIN_EPOCHS,
                                          batch_size=64,
                                          device='cuda',
                                          early_stopping_patience=5,
                                          max_batches_per_epoch_train=500,
                                          max_batches_per_epoch_val=100,
                                          lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                  factor=0.5,
                                                                                                                  verbose=True))
    
    if not QUICK_RUN:
        torch.save(single_token_model.state_dict(), './models/single_token_pos.pth')                                                                                                                    

Эпоха 0
Эпоха: 20 итераций, 3.06 сек
Среднее значение функции потерь на обучении 1.0005273409187794
Среднее значение функции потерь на валидации 0.5056548970086234
Новая лучшая модель!



Смотрим что получилось

- распределение классов сильно неравномерное, поэтому accuracy нас совсем не интересует
- т.к. классов много, то смотрим интегральные показатели `macro avg` - тут все прилично
  - на обучающей и валидационной выборках порядка **0.87-0.89**

In [25]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
TEST_UNIQUE_TAGS = np.array(UNIQUE_TAGS)[np.unique(test_labels)].tolist()   # классов в тесте может быть меньше
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=TEST_UNIQUE_TAGS))

39it [00:01, 37.97it/s]                             


Среднее значение функции потерь на обучении 0.2912595868110657
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00     68557
         ADJ       0.37      0.47      0.41      1349
         ADP       0.70      0.80      0.75      1638
         ADV       0.27      0.09      0.14      1021
         AUX       0.00      0.00      0.00       125
       CCONJ       0.87      0.60      0.71       641
         DET       0.00      0.00      0.00       404
        INTJ       0.00      0.00      0.00         7
        NOUN       0.57      0.62      0.59      3925
         NUM       0.00      0.00      0.00       174
        PART       0.93      0.45      0.60       451
        PRON       0.48      0.53      0.51       912
       PROPN       0.08      0.00      0.00       414
       PUNCT       0.96      0.98      0.97      3538
       SCONJ       0.39      0.52      0.45       257
         SYM       0.00      0.00      0.00         2
        VERB      

101%|██████████| 14/13.90625 [00:00<00:00, 37.72it/s]


Среднее значение функции потерь на валидации 0.5066335201263428
              precision    recall  f1-score   support

     <NOTAG>       0.99      1.00      1.00     22593
         ADJ       0.49      0.50      0.50       895
         ADP       0.61      0.79      0.69       875
         ADV       0.13      0.09      0.11       275
         AUX       0.00      0.00      0.00        58
       CCONJ       0.82      0.69      0.75       224
         DET       0.00      0.00      0.00       145
        NOUN       0.66      0.64      0.65      2513
         NUM       0.00      0.00      0.00       299
        PART       0.91      0.29      0.44       142
        PRON       0.31      0.55      0.39       170
       PROPN       1.00      0.00      0.01       590
       PUNCT       0.84      0.90      0.87      1574
       SCONJ       0.23      0.25      0.24        85
        VERB       0.30      0.69      0.42       652
           X       0.00      0.00      0.00        60

    accuracy    

In [26]:
del train_pred, train_loss, test_pred, test_loss    # больше не нужны
gc.collect()

0

## Предсказание частей речи на уровне предложений (с учётом контекста)

В `forward` тоже самое, кроме:

1. эмбеддинги такие же (тоже схлопываем измерение предложений!)
2. backbone нейромодуль такой же (просто перименован в single_token_backbone)
3. пулинг такой же

Дальше отличия - проделываем примерно тоже самое что и с символами в токене, но уже на уровне токенов в предложении. 

Для этого:
- добавлен еще один сверточный модуль `context_backbone` (архитектура идентичная, со "skip connections")
- выходной модуль `out` это не линейный классификатор (логит), а еще одна свертка `nn.Conv1d`
  
4. Сверточный модуль: 
    - возвращаем измерение предложений, меняем порядок измерений под требования торча
    - передаем на свертку
    - получаем тоже 3д тензор, той же размерности, но теперь тензоры токенов учитывают положение токена в предложении
5. Принятие решения о классе:
   - это **одномерна свертка с ядром размера 1** (окно ядра это один токен)
   - она проецирует признаки (вектор/тензор) каждого токена на пространство классов

Физический смысл того, что мы описали сейчас, заключается в том, чтобы сначала проанализировать структуру каждого слова, найти там какие-то суффиксы и окончания (пп. 1-3) а затем — смешать информацию о структуре каждого слова с контекстом, в котором это слово употребляется (п. 4). 

\*) Почему до этого **был полносвязный слой (линейный мультиклассификатор), а теперь одномерная свертка с ядром 1**? 

Разницы нет, применить к матрице $A$ размера $C_{in} \times L_{in}$ (в нашем случае число каналов — это размерность вектора эмбеддинга, а длина последовательности — это максимальная длина предложения) одномерную свертку $K$ c числом выходных каналов $C_{out}$ — это то же, что представить $K$ как матрицу размера $C_{in} \times C_{out}$ и умножить $A^T$ на $K$ (в pytorch в `Conv1d` размерности расставлены в порядке, обратном тому, который мы брали ранее).

In [27]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

Количество параметров почти удвоилось (84 тыс.), т.к.:
- добавился набор параметров для анализа контекста токена

In [28]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 80338


На времени обучения это не сказывается, т.к. количество параметров съедает память, а не время.

In [29]:
if LOAD_LAST_MODEL:
    try:
        msg = sentence_level_model.load_state_dict(torch.load('./models/sentence_level_pos.pth'))
        print("Loading pretrained: ", msg)
    except RuntimeError as e:
        print("Конфигурация сети поменялась, сохраненная модель не подходит")
        raise e
else:
    torch.cuda.empty_cache()

    (val_loss,
    sentence_level_model) = train_eval_loop(sentence_level_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=NUM_TRAIN_EPOCHS,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                        factor=0.5,
                                                                                                                        verbose=True))
    if not QUICK_RUN:
        torch.save(sentence_level_model.state_dict(), './models/sentence_level_pos.pth')                                                                                                                         

Эпоха 0
Эпоха: 20 итераций, 2.56 сек
Среднее значение функции потерь на обучении 0.8832546755671501
Среднее значение функции потерь на валидации 0.4321576399462564
Новая лучшая модель!



Немного улучшились метрики, но не кардинально.

In [30]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
TEST_UNIQUE_TAGS = np.array(UNIQUE_TAGS)[np.unique(test_labels)].tolist()   # классов в тесте может быть меньше
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=TEST_UNIQUE_TAGS))

39it [00:01, 36.61it/s]                             


Среднее значение функции потерь на обучении 0.2532041668891907
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00     68557
         ADJ       0.33      0.44      0.38      1349
         ADP       0.91      0.72      0.81      1638
         ADV       0.30      0.09      0.14      1021
         AUX       0.00      0.00      0.00       125
       CCONJ       0.81      0.62      0.70       641
         DET       0.10      0.00      0.00       404
        INTJ       0.00      0.00      0.00         7
        NOUN       0.47      0.84      0.60      3925
         NUM       0.00      0.00      0.00       174
        PART       0.94      0.45      0.61       451
        PRON       0.84      0.27      0.41       912
       PROPN       0.00      0.00      0.00       414
       PUNCT       0.95      1.00      0.97      3538
       SCONJ       0.60      0.41      0.49       257
         SYM       0.00      0.00      0.00         2
        VERB      

101%|██████████| 14/13.90625 [00:00<00:00, 36.06it/s]


Среднее значение функции потерь на валидации 0.433011919260025
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00     22593
         ADJ       0.41      0.44      0.43       895
         ADP       0.76      0.72      0.74       875
         ADV       0.20      0.08      0.11       275
         AUX       0.00      0.00      0.00        58
       CCONJ       0.77      0.69      0.73       224
         DET       0.50      0.01      0.01       145
        NOUN       0.55      0.77      0.64      2513
         NUM       0.00      0.00      0.00       299
        PART       0.95      0.29      0.44       142
        PRON       0.59      0.19      0.29       170
       PROPN       0.00      0.00      0.00       590
       PUNCT       0.83      0.93      0.88      1574
       SCONJ       0.40      0.19      0.26        85
        VERB       0.31      0.46      0.37       652
           X       0.00      0.00      0.00        60

    accuracy     

In [31]:
del train_pred, train_loss, test_pred, test_loss    # больше не надо
gc.collect()

0

## Применение полученных теггеров и сравнение

Сделан отдельный класс для расстановки тегов части речи `POSTagger`:
- принимает на вход обученную модель
- отображение символов в ИД
- отображение номеров тегов в строковое представление
- статистику корпуса
- единственный метод `__call__`

Чтобы определить части речи токенов, сначала мы токенизируем переданное нам предложение, затем делаем то, что мы делали при обучении — а именно, перекладываем информацию о символах в тензор, затем, в цикле, применяем нашу модель для каждого предложения. В результате применения нашей модели к каждому предложению мы получаем трёхмерный тензор, первая размерность этого тензора соответствует количеству предложений, вторая — это количество меток, а третья — это максимальная длина предложения.

Таким образом, для каждого предложения и для каждого токена у нас есть распределение вероятностей по меткам. И здесь мы просто выбираем для каждого токена наиболее вероятную метку. 

Ну, и напоследок — мы преобразовываем полученную информацию в формат, удобный для человека, то есть мы преобразовываем номера тэгов в их строковое название.

In [32]:
single_token_pos_tagger = POSTagger(single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [33]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок',
    'Он видел их семью своими глазами',
    'Эти типы стали есть в цехе',
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

Без контекста получилось вот:
- а вот вообще то нормально получилось то, че вы сразу

In [34]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))

1it [00:00, 91.86it/s]                     

мама-NOUN мыла-VERB раму-NOUN
косил-VERB косой-ADJ косой-ADJ косой-ADJ
глокая-VERB куздра-NOUN штеко-ADJ будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN
сяпала-VERB калуша-VERB с-ADP калушатами-NOUN по-ADP напушке-ADV
пирожки-NOUN поставлены-VERB в-ADP печь-VERB мама-NOUN любит-VERB печь-VERB
ведро-NOUN дало-VERB течь-VERB вода-ADP стала-VERB течь-VERB
три-NOUN да-ADP три-NOUN будет-NOUN дырка-NOUN
три-NOUN да-ADP три-NOUN будет-NOUN шесть-VERB
сорок-ADJ сорок-ADJ
он-PRON видел-NOUN их-CCONJ семью-ADJ своими-NOUN глазами-NOUN
эти-NOUN типы-NOUN стали-VERB есть-VERB в-ADP цехе-ADJ


С контекстом получилось вот:
- не везде, но кое где сработало
- сильно хуже вышло, чем было у авторов

Причины не срабатывания помимо "просто недоучили сеть" ("мама мыла раму"):
- "три/три" - определяющее контекстное слово "дырка/шесть" стоит дальше чем размер ядра и сеть его не видит, нужно более крупное ядро или больше слоев сверток

In [35]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))

1it [00:00, 86.07it/s]                     


мама-NOUN мыла-VERB раму-NOUN
косил-VERB косой-ADJ косой-ADJ косой-NOUN
глокая-NOUN куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-NOUN бокрёнка-NOUN
сяпала-VERB калуша-VERB с-ADP калушатами-NOUN по-ADP напушке-NOUN
пирожки-NOUN поставлены-NOUN в-ADP печь-NOUN мама-VERB любит-VERB печь-NOUN
ведро-NOUN дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN
три-NOUN да-VERB три-NOUN будет-NOUN дырка-NOUN
три-NOUN да-VERB три-NOUN будет-NOUN шесть-NOUN
сорок-NOUN сорок-NOUN
он-PRON видел-NOUN их-CCONJ семью-NOUN своими-NOUN глазами-NOUN
эти-NOUN типы-NOUN стали-VERB есть-NOUN в-ADP цехе-NOUN


## Свёрточный модуль своими руками

Все pytorch-модули наследуются от базового класса `nn.Module`.

Реализуем не весь, но основной функционал стандартного модуля `nn.Conv1d` (так, чтоб он мог подключиться в конвеер вместо оригинального):
- чтобы детально посмотреть механизм работы, как учитываются входные данные, как обрабатываются отдельные каналы (признаки) и т.п.

Одномерные свертки **принимают на вход** 3д тензора (размер батча, количество входных каналов, длина одномерной последовательности)

Одномерные свертки **возвращают** также 3д тензор размерности ["количество элементов в батче", "количество выходных каналов", "новая длина последовательности"]. Длина последовательности может либо остаться прежней, либо уменьшится, либо увеличиться — это зависит от размеров ядра и от паддинга.

`__init__`

- сохраняем входные параметры
- создаем начальный тензор **весов**, он же **ядро свертки** (инициируем нормальным (почему?) шумом с малой дисперсией) 
  - количество строк равно количеству входных каналов умноженному на размер свёртки (на размер ядра), а количество столбцов этой матрицы равно количеству выходных каналов
- создаем начальный тензор смещений (инициируем нулями)
  - размерость по количеству выходных каналов

`forward`

1. Делаем паддинг
- увеличиваем длину 3-го измерения на паддинг
- заполняем их нулями
- обновляем переменную с размером 3-го измерения (используется дальше)

2. Подготовка признаков
- это матрица окон, по которым проходи ядро слепленных вертикально (torch.cat(chunks, dim=1))

3. Транспонируем под требования торча и применяем ядро

- `torch.bmm` (batch matrix multiplication) - на вход идут
  - матрица признаков, которая берется построчно (по окнам)
  - ядро свертки (веса) - которые надо дополнить измерениями для совместности матричного умножения
    - expand изменяет размер тензора, но, при этом, она делает это без выделения дополнительной памяти, то есть снаружи выглядит, что тензор большой, а на самом деле это — просто плоская матрица. Похоже просто переиндексация
- добавляются смещения self.bias тоже дополненные измерениями для тензорной совместности 
- транспонируем под сигнатуру входного тензора

Все просто, если четко соблюдать размерности и порядки измерений.

In [36]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, **kwargs):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""
        
        # тут делается паддинг
        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)    # concat в торче
            sequence_len = x.shape[-1]
        
        # тут подготовка признаков
        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

Количество параметров такое же

In [37]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 80338


У авторов эта реализация требовала 50 сек. для расчета эпохи.

Эта разница имеет значение только для той версии pytorch, которую используем мы сейчас, в другой версии pytorch разница может быть совершенно другая. 

Наиболее вероятно, что это ускорение вызвано тем, что наша реализация свёрток узкоспециализирована, в ней нету кучи разных вариантов паддинга, в ней нет механизма прореживания ядра, а также нет страйдов — в ней нельзя задавать шаг, с которым нужно идти скользящим окном по исходной последовательности. Другими словами, она проще гораздо, чем стандартная реализация свёрток. 

Таким образом, иногда имеет смысл что-то реализовать самому, но надо помнить, что в машинном обучении, часто, гораздо важнее быстро проверять разные гипотезы, а для этого лучше использовать стандартные модули, которые проверены, работают надёжно в разных ситуациях, они универсальны и вам не нужно тратить время на написание своих модулей. 

Свои модули имеет смысл писать только тогда, когда вы точно знаете ту архитектуру, которая вам нужна для решения вашей прикладной задачи, и вы хотите её ускорить. Например — для того, чтобы уметь запускать её на мобильном телефоне.

In [38]:
if LOAD_LAST_MODEL:
    # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
    try:
        msg = sentence_level_model_my_conv.load_state_dict(torch.load('./models/sentence_level_model_my_conv.pth'))
        print("Loading pretrained: ", msg)
    except RuntimeError as e:
        print("Конфигурация сети поменялась, сохраненная модель не подходит")
        raise e
else:
    torch.cuda.empty_cache()

    (val_loss,
    sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                    train_dataset,
                                                    test_dataset,
                                                    F.cross_entropy,
                                                    lr=5e-3,
                                                    epoch_n=NUM_TRAIN_EPOCHS,
                                                    batch_size=24, # 64 не дает
                                                    device='cuda',
                                                    early_stopping_patience=5,
                                                    max_batches_per_epoch_train=500,
                                                    max_batches_per_epoch_val=100,
                                                    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                factor=0.5,
                                                                                                                                verbose=True))
    if not QUICK_RUN:
        torch.save(sentence_level_model_my_conv.state_dict(), './models/sentence_level_model_my_conv.pth')

Эпоха 0
Эпоха: 52 итераций, 4.83 сек
Среднее значение функции потерь на обучении 0.6565444871353415
Среднее значение функции потерь на валидации 0.2849828514613603
Новая лучшая модель!



Качество не хуже, даже получше.

In [39]:
train_pred = predict_with_model(sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
TEST_UNIQUE_TAGS = np.array(UNIQUE_TAGS)[np.unique(test_labels)].tolist()   # классов в тесте может быть меньше
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=TEST_UNIQUE_TAGS))

39it [00:01, 23.33it/s]                             


Среднее значение функции потерь на обучении 0.13216552138328552
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00     68557
         ADJ       0.70      0.56      0.63      1349
         ADP       0.89      0.92      0.90      1638
         ADV       0.63      0.16      0.26      1021
         AUX       0.80      0.77      0.78       125
       CCONJ       0.90      0.84      0.87       641
         DET       0.58      0.45      0.51       404
        INTJ       0.00      0.00      0.00         7
        NOUN       0.64      0.89      0.74      3925
         NUM       0.82      0.24      0.37       174
        PART       0.75      0.63      0.68       451
        PRON       0.85      0.52      0.65       912
       PROPN       0.86      0.61      0.71       414
       PUNCT       1.00      1.00      1.00      3538
       SCONJ       0.69      0.70      0.70       257
         SYM       0.00      0.00      0.00         2
        VERB     

101%|██████████| 14/13.90625 [00:00<00:00, 23.33it/s]

Среднее значение функции потерь на валидации 0.2861635386943817
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00     22593
         ADJ       0.72      0.58      0.64       895
         ADP       0.84      0.90      0.87       875
         ADV       0.19      0.05      0.09       275
         AUX       0.86      0.88      0.87        58
       CCONJ       0.80      0.77      0.78       224
         DET       0.53      0.46      0.49       145
        NOUN       0.66      0.83      0.74      2513
         NUM       0.94      0.55      0.70       299
        PART       0.73      0.46      0.57       142
        PRON       0.60      0.42      0.49       170
       PROPN       0.68      0.16      0.26       590
       PUNCT       0.92      0.98      0.95      1574
       SCONJ       0.63      0.47      0.54        85
        VERB       0.49      0.73      0.59       652
           X       0.00      0.00      0.00        60

    accuracy    

In [40]:
del train_pred, train_loss, test_pred, test_loss    # больше не нужны
gc.collect()

0

In [41]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |     802 KB |  292221 KB |  142576 MB |  142575 MB |
|       from large pool |       0 KB |  289280 KB |  139169 MB |  139169 MB |
|       from small pool |     802 KB |    8723 KB |    3406 MB |    3406 MB |
|---------------------------------------------------------------------------|
| Active memory         |     802 KB |  292221 KB |  142576 MB |  142575 MB |
|       from large pool |       0 KB |  289280 KB |  139169 MB |

In [42]:
print("Number of unreachable objects collected by GC:", gc.collect())

Number of unreachable objects collected by GC: 112


# До.за.
 
В качестве домашнего задания мы предлагаем Вам поэкспериментировать с кодом этого семинара, чтобы лучше понять особенности свёрточных нейросетей и попробовать улучшить качество определения частей речи. Что можно попробовать сделать:

- поиграться с параметрами и архитектурой - количеством каналов (размерностью эмбеддинга), глубиной нейросети, силой Dropout, добавить BatchNorm или другую нормализацию
- подключить прореженные (dilated) свёртки, чтобы увеличить рецептивное поле без увеличения числа параметров
- добавить взвешивание классов
- использовать в качестве обозначения начала и конца слова не 0, а какой-нибудь другой токен (для 0 nn.Embedding всегда выдаёт - нулевой вектор, а в этом случае для начала а конца слова будут учиться специальные вектора)

Также мы предлагаем Вам не ограничиваться этим списком, а придумать свои способы улучшить качество определения частей речи.

**Заново соберем все данные**

In [43]:
def prepate_data():    
    full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
    full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

    MAX_SENT_LEN = max(max(len(sent) for sent in full_train), max(len(sent) for sent in full_test)) 
    MAX_ORIG_TOKEN_LEN = max(max(len(token.form) for sent in full_train for token in sent), 
                            max(len(token.form or [0]) for sent in full_test for token in sent))

    all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]

    train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
    char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')

    UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
    label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

    train_inputs, train_labels, err1 = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
    train_dataset = TensorDataset(train_inputs, train_labels)

    test_inputs, test_labels, err2 = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
    test_dataset = TensorDataset(test_inputs, test_labels)

    del train_char_tokenized, full_train, full_test, train_inputs, test_inputs
    gc.collect()

    return char_vocab, label2id, train_dataset, test_dataset

# char_vocab, label2id, train_dataset, test_dataset = prepate_data()

**Добавили разреженное ядро (`dilation = 2`)**
- остальное не меняем
- время эпохи увеличилось на 10%

In [44]:
torch.backends.cudnn.deterministic=False        # пока неясно зачем, но так рекомендуют для dilation сверток

sentence_level_model_homework = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.1, dilation=2),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.1))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_homework.parameters()))

Количество параметров 80338


In [45]:
if LOAD_LAST_MODEL:
    try:
        msg = sentence_level_model_homework.load_state_dict(torch.load('./models/sentence_level_model_homework.pth'))
        print("Loading pretrained: ", msg)
    except RuntimeError as e:
        print("Конфигурация сети поменялась, сохраненная модель не подходит")
        raise e
else:
    torch.cuda.empty_cache()

    (val_loss,
    sentence_level_model_homework) = train_eval_loop(sentence_level_model_homework,
                                                    train_dataset,
                                                    test_dataset,
                                                    F.cross_entropy,
                                                    lr=5e-3,
                                                    epoch_n=NUM_TRAIN_EPOCHS,
                                                    batch_size=64,
                                                    device='cuda',
                                                    early_stopping_patience=5,
                                                    max_batches_per_epoch_train=500,
                                                    max_batches_per_epoch_val=100,
                                                    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                factor=0.5,
                                                                                                                                verbose=True))
    if not QUICK_RUN:
        torch.save(sentence_level_model_homework.state_dict(), './models/sentence_level_model_homework.pth')   

Эпоха 0
Эпоха: 20 итераций, 2.45 сек
Среднее значение функции потерь на обучении 1.0558781012892724
Среднее значение функции потерь на валидации 0.4848137540476663
Новая лучшая модель!



**Метрики:**
- на трейне улучшились 0,93
- на тесте не поменялись 0,89

In [46]:
train_pred = predict_with_model(sentence_level_model_homework, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model_homework, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
TEST_UNIQUE_TAGS = np.array(UNIQUE_TAGS)[np.unique(test_labels)].tolist()   # классов в тесте может быть меньше
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=TEST_UNIQUE_TAGS))

39it [00:01, 36.49it/s]                             


Среднее значение функции потерь на обучении 0.2614191770553589
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00     68557
         ADJ       0.58      0.34      0.43      1349
         ADP       0.89      0.64      0.75      1638
         ADV       0.27      0.15      0.20      1021
         AUX       0.00      0.00      0.00       125
       CCONJ       0.67      0.67      0.67       641
         DET       0.00      0.00      0.00       404
        INTJ       0.00      0.00      0.00         7
        NOUN       0.47      0.83      0.60      3925
         NUM       0.00      0.00      0.00       174
        PART       0.82      0.48      0.60       451
        PRON       0.54      0.32      0.40       912
       PROPN       0.00      0.00      0.00       414
       PUNCT       0.96      1.00      0.98      3538
       SCONJ       0.00      0.00      0.00       257
         SYM       0.00      0.00      0.00         2
        VERB      

101%|██████████| 14/13.90625 [00:00<00:00, 36.27it/s]


Среднее значение функции потерь на валидации 0.485891193151474
              precision    recall  f1-score   support

     <NOTAG>       0.99      1.00      1.00     22593
         ADJ       0.59      0.35      0.44       895
         ADP       0.87      0.62      0.73       875
         ADV       0.11      0.07      0.09       275
         AUX       0.00      0.00      0.00        58
       CCONJ       0.58      0.73      0.64       224
         DET       0.00      0.00      0.00       145
        NOUN       0.54      0.79      0.64      2513
         NUM       0.00      0.00      0.00       299
        PART       0.84      0.37      0.52       142
        PRON       0.42      0.35      0.38       170
       PROPN       0.00      0.00      0.00       590
       PUNCT       0.81      0.93      0.87      1574
       SCONJ       0.00      0.00      0.00        85
        VERB       0.31      0.51      0.39       652
           X       0.00      0.00      0.00        60

    accuracy     

**Заковыристые тестовые фразы:**
- распознавание не улучшилось

In [47]:
sentence_level_pos_tagger_dilation = POSTagger(sentence_level_model_homework, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger_dilation(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 83.72it/s]                     

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-NOUN косой-NOUN

глокая-NOUN куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-NOUN бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-NOUN в-ADP печь-NOUN мама-NOUN любит-NOUN печь-NOUN

ведро-VERB дало-VERB течь-NOUN вода-VERB стала-VERB течь-NOUN

три-NOUN да-NOUN три-NOUN будет-NOUN дырка-NOUN

три-NOUN да-NOUN три-NOUN будет-NOUN шесть-NOUN

сорок-NOUN сорок-NOUN

он-PRON видел-VERB их-CCONJ семью-NOUN своими-NOUN глазами-NOUN

эти-VERB типы-VERB стали-VERB есть-NOUN в-ADP цехе-NOUN

